### Загрузка данных

In [ ]:
import pandas as pd

# чтение файлов
train_df = pd.read_parquet('train.parquet')
test_df = pd.read_parquet('test.parquet')

# отбросим столбец id
train_df.drop(["id"], axis=1, inplace=True)

In [ ]:
train_df['okved'] = train_df['okved'].astype('category')

## Обработка данных

### Обработка категориальных признаков

In [ ]:
cat_cols = [
    'channel_code', 'city', 'city_type',
    'index_city_code', 'ogrn_month', 'ogrn_year',
    'branch_code', 'okved', 'segment'
]

train_df[cat_cols] = train_df[cat_cols].astype('category')
test_df[cat_cols] = test_df[cat_cols].astype('category')

### Обработка пустых значений

### Признаки с малым кол-вом пропусков заменяем медианой или наиболее часто встречающимся значением

Заметим, что в этих признаках процент пропусков составляет от 86-93%
Поэтому просто отбросим эти признаки, а признак регистрации ОГРН сделаем бинарным - зарегестрировано/незарегистрированно


In [ ]:
train_df.drop(["max_end_plan_non_fin_deals",
               "min_end_plan_non_fin_deals",
               "max_start_non_fin_deals",
               "min_start_non_fin_deals",
               "max_end_fact_fin_deals",
               "max_start_fin_deals",
               "min_end_fact_fin_deals",
               "min_start_fin_deals",
               ], axis=1, inplace=True)

test_df.drop(["max_end_plan_non_fin_deals",
              "min_end_plan_non_fin_deals",
              "max_start_non_fin_deals",
              "min_start_non_fin_deals",
              "max_end_fact_fin_deals",
              "max_start_fin_deals",
              "min_end_fact_fin_deals",
              "min_start_fin_deals",
              ], axis=1, inplace=True)

Если значение min(max)_founderpres - NaN значит человек не регистрировал себе ОГРН, иначе регистрировал.

In [ ]:
train_df["ogrn_reg"] = (train_df["max_founderpres"].isna()*1 - 1) * -1
train_df[["ogrn_reg", "max_founderpres"]]

Проделаем тоже самое для тестовых данных

In [ ]:
test_df["ogrn_reg"] = (test_df["max_founderpres"].isna()*1 - 1) * -1
test_df.drop(["max_founderpres", "min_founderpres"], axis=1, inplace=True)

train_df["ogrn_reg"] = train_df["ogrn_reg"].astype("category")
test_df["ogrn_reg"] = test_df["ogrn_reg"].astype("category")
# train_df[cat_indexes] = train_df[cat_indexes].astype("category")
# test_df[cat_indexes] = test_df[cat_indexes].astype("category")

Проверим, остались ли пропуски после обработки

В данных есть признак - index_city_code	код города в почтовом индексе. Так как мы закодировали города, можно избавиться от этого признака, он будет создавать лишние зависимости

In [ ]:
train_df.drop(['max_founderpres', 'min_founderpres'], axis=1, inplace=True)

In [ ]:
train_df.drop(['index_city_code'], axis=1, inplace=True)

# повторим для тестовых данных
test_df.drop(['index_city_code'], axis=1, inplace=True)

Заметим, что в наших данных есть такие признаки как sum_a_oper_1m и cnt_a_oper_1m - сумма операций типа А за месяц и их количество. Это распространяется и на другие типы операций. Сконструируем новый признак sum/cnt - средняя сумма операции типа A в месяц. Распространим эту логику на все типы операций.

In [ ]:
train_nans = train_df.isna().sum().sort_values(ascending=False).loc[lambda x: x > 0]
#в обучающих данных
train_nans = train_nans.loc[lambda x: x > 0]
smaller_train_nans = train_nans

for i in smaller_train_nans.index:
    if train_df[i].dtype == "object" or train_df[i].dtype == "category":
        train_df[i].loc[train_df[i].isna()] = train_df[i].value_counts().sort_values(ascending=False).index[0]
    else:
        train_df[i].loc[train_df[i].isna()] = train_df[i].median()


# в тестовых данных
test_nans = test_df.isna().sum().sort_values(ascending=False).loc[lambda x: x > 0]

smaller_test_nans = test_nans

for i in smaller_test_nans.index:
    if test_df[i].dtype == "category":
        test_df[i].loc[test_df[i].isna()] = test_df[i].value_counts().sort_values(ascending=False).index[0]
    else:
        test_df[i].loc[test_df[i].isna()] = test_df[i].median()

test_nans = test_df.isna().sum().sort_values(ascending=False)

Новые фичи


In [ ]:
# растет ли сумма операций в течении времени
check_df = pd.DataFrame()
check_df['deb_e_oper_growth'] = train_df['sum_deb_e_oper_1m'] > train_df['sum_deb_e_oper_3m']
check_df['deb_f_oper_growth'] = train_df['sum_deb_f_oper_1m'] > train_df['sum_deb_f_oper_3m']
check_df['deb_h_oper_growth'] = train_df['sum_deb_h_oper_1m'] > train_df['sum_deb_h_oper_3m']

check_df['cred_e_oper_growth'] = train_df['sum_cred_e_oper_1m'] > train_df['sum_cred_e_oper_3m']
check_df['cred_f_oper_growth'] = train_df['sum_cred_f_oper_1m'] > train_df['sum_cred_f_oper_3m']
check_df['cred_h_oper_growth'] = train_df['sum_cred_h_oper_1m'] > train_df['sum_cred_h_oper_3m']

check_df[['deb_e_oper_growth', 'deb_f_oper_growth', 'deb_h_oper_growth']] = check_df[['deb_e_oper_growth', 'deb_f_oper_growth', 'deb_h_oper_growth']].astype('category')
check_df[['cred_e_oper_growth', 'cred_f_oper_growth', 'cred_h_oper_growth']] = check_df[['cred_e_oper_growth', 'cred_f_oper_growth', 'cred_h_oper_growth']].astype('category')
dumies = pd.get_dummies(train_df, columns=['segment'])
check_df = check_df.join(dumies)
train_df = check_df


check_df = pd.DataFrame()
check_df['deb_e_oper_growth'] = test_df['sum_deb_e_oper_1m'] > test_df['sum_deb_e_oper_3m']
check_df['deb_f_oper_growth'] = test_df['sum_deb_f_oper_1m'] > test_df['sum_deb_f_oper_3m']
check_df['deb_h_oper_growth'] = test_df['sum_deb_h_oper_1m'] > test_df['sum_deb_h_oper_3m']

check_df['cred_e_oper_growth'] = test_df['sum_cred_e_oper_1m'] > test_df['sum_cred_e_oper_3m']
check_df['cred_f_oper_growth'] = test_df['sum_cred_f_oper_1m'] > test_df['sum_cred_f_oper_3m']
check_df['cred_h_oper_growth'] = test_df['sum_cred_h_oper_1m'] > test_df['sum_cred_h_oper_3m']

check_df[['deb_e_oper_growth', 'deb_f_oper_growth', 'deb_h_oper_growth']] = check_df[['deb_e_oper_growth', 'deb_f_oper_growth', 'deb_h_oper_growth']].astype('category')
check_df[['cred_e_oper_growth', 'cred_f_oper_growth', 'cred_h_oper_growth']] = check_df[['cred_e_oper_growth', 'cred_f_oper_growth', 'cred_h_oper_growth']].astype('category')

dumies = pd.get_dummies(test_df, columns=['segment'])
check_df = check_df.join(dumies)
test_df = check_df

In [ ]:
train_df.drop(['index_city_code'], axis=1, inplace=True)
test_df.drop(['index_city_code'], axis=1, inplace=True)

In [ ]:
# средняя сумма операций различного типа
def med_sum_oper(df):
    new_features = ['med_sum_oper_a', 'med_sum_oper_b', 
                    'med_sum_oper_c', 'med_sum_oper_d',
                    'med_sum_oper_e','med_sum_oper_f',
                    'med_sum_oper_g','med_sum_oper_h']
    sum_oper =  [
                ('sum_a_oper', 'cnt_a_oper'),
                ('sum_b_oper', 'cnt_b_oper'),
                ('sum_c_oper', 'cnt_c_oper'),
                ('sum_d_oper', 'cnt_d_oper'),
                ('sum_e_oper', 'cnt_e_oper'),
                ('sum_f_oper', 'cnt_f_oper'),
                ('sum_g_oper', 'cnt_g_oper'),
                ('sum_h_oper', 'cnt_h_oper'),
            ]

    for new_feature, origin_features in zip(new_features, sum_oper):
        df[new_feature+'_1m'] = df[origin_features[0]+'_1m']/df[origin_features[1]+'_1m']
        df[new_feature+'_3m'] = df[origin_features[0]+'_3m']/df[origin_features[1]+'_3m']
    
    return df

train_df = med_sum_oper(train_df)
test_df = med_sum_oper(test_df)

Обработаем выбросы с помощью изолированного дерева

In [ ]:
from sklearn.ensemble import IsolationForest

# Выберем признаки, в которых не будет происходить поиск аномалий (категориальные признаки)
cat_cols = [
    'channel_code', 'city', 'city_type', 'ogrn_month', 'ogrn_year',
    'branch_code', 'okved', 'segment'
]

# Отфильтруем категориальные признаки из всех признаков
no_anomaly_features = [col for col in train_df.columns if col not in cat_cols]


df_to_filter = train_df[no_anomaly_features]
iso_clf = IsolationForest(random_state=0, contamination=0.2, bootstrap=True).fit(df_to_filter)
anomaly = iso_clf.predict(df_to_filter)

# Отфильтруем только те строки, которые не являются аномалиями
train_df_filtered = train_df[anomaly == 1]

# Присоединим отфильтрованные числовые данные к категориальным данным
train_df_filtered_with_categorical = train_df_filtered[cat_cols].join(train_df_filtered[no_anomaly_features])
print(train_df_filtered_with_categorical.shape)
train_df = train_df_filtered_with_categorical

Теперь определим меру взаимной информации для признаков, чтобы понять какие из признаков действительно оказывают влияние на модель

In [ ]:
from sklearn.feature_selection import mutual_info_classif
def mutual_inform_sorted(target):
    '''
    Определение взаимной информации для признаков, 
    чтобы понять какие из признаков действительно оказывают влияние на модель
    '''
    y = target.to_numpy().ravel()
    mi = mutual_info_classif(X, y)
    mi_sc = pd.Series(mi, name="MI Scores", index=X.columns)
    mi_sc = mi_sc.sort_values(ascending=False)
    
    return mi_sc


names = [column for column in train_df if column not in ['target_1', 'target_2', 'total_target']]
X = train_df[names[:90]]

mi_sc_1 = mutual_inform_sorted(train_df["target_1"])
mi_sc_2 = mutual_inform_sorted(train_df['target_2'])


In [ ]:
# удалим наименее влиятельные признаки
zero_mi_cols_1 = mi_sc_1[mi_sc_1 <= 0.015].index.tolist()
zero_mi_cols_2 = mi_sc_2[mi_sc_2 <= 0.015].index.tolist()

X1_train = train_df.drop(columns=zero_mi_cols_1)
X1_test = test_df.drop(columns=zero_mi_cols_1)

X2_train = train_df.drop(columns=zero_mi_cols_2)
X2_test = test_df.drop(columns=zero_mi_cols_2)

In [ ]:
datasets = [(X1_train, X1_test), (X2_train, X2_test)]
for i, elem in enumerate(datasets):
    elem[0].to_csv(f'./Data/target_{i+1}_train', index=False)
    elem[1].to_csv(f'./Data/target_{i+1}_test', index=False)